# Tutorial: Perturbation of the neoclassical model

## Quantecon workshop - Chilean Central Bank - 2022

__Warm-up: install and test the `ForwardDiff` library. Check the jacobian function.__

__Warm-up(2): install and test the `NLSolve` library. Find the root of a simple 2 variables function.__

__Create a named tuple to hold the model parameters__

In [7]:
p0 = let
    α = 0.3  # capital share (from the data)
    β = 0.96 # time discount  (calibrated to 1/(1+r), where r=α4%)
    γ = 4.0  # risk aversion or 1/EIS. EIS is calibrated from macro models
    δ = 0.1   # capital depreciation (from the data)
    ρ = 0.9   # autocorrelation of productivity shock
    Σ = [0.01;]  # covariance matrix of innovations

    p = (;α,β,γ,δ,ρ,σ)   # named tuple of parameters
end
Calibration = typeof(p0)

NamedTuple{(:α, :β, :γ, :δ, :ρ, :σ), NTuple{6, Float64}}

__Define two functions:__
- `transition(z::Number, k::Number, i::Number, c::Number, ϵ::Number, p::Calibration)::Vector{Number}` which returns productivity and capital at date `t+1` as a function of productivity, capital, consumption, investment at date `t`
- `arbitrage(z::Number, k::Number, i::Number, Z::Number, K::Number, I::Number, p::Calibration)::Vector{Number}` which returns the residual of the euler equation (lower case variable for date t, upper case for date t+1)


In [8]:
function transition(z::Number, k::Number, i::Number, c::Number, ϵ::Number, p::Calibration)
    # Z = 
    # K = 
    return [Z, K]
end

transition (generic function with 1 method)

In [9]:
function arbitrage(z::Number, k::Number, i::Number, c::Number,
                   Z::Number, K::Number, I::Number, C::Number, p::Calibration)
    

    y = exp(z)*k^p.α

    eq = c - y + i
    optimality = p.β*(C/c)^(-p.γ)*(1-p.δ + p.α*exp(Z)*K^(p.α-1) ) - 1

    return [eq, optimality]

end

arbitrage (generic function with 1 method)

__Using multiple dispatch, define two variants of the same functions, that take vectors as input and output arguments:__
- `arbitrage(s::Vector{Number}, x::Vector{Number}, S::Vector{Number}, X::Vector{Number}, p::Calibration)`
- `transition(s::Vector{Number}, x::Vector{Number}, e::Vector{Number}, p::Calibration)`

In [22]:
transition(s, x, e, p) = transition(s[1],s[2],x[1],x[2],e[1],p)
arbitrage(s, x, S, X, p) = arbitrage(s[1],s[2],x[1],x[2],S[1],S[2],X[1],X[2],p)

arbitrage (generic function with 2 methods)

In [23]:
# let's check it works:
transition([0.1, 4.0], [0.4, 0.2], [0.01], p0)
arbitrage([0.1, 4.0], [0.4, 0.2], [0.1, 4.0], [0.4, 0.1], p0)

UndefVarError: UndefVarError: Z not defined

__Write a function `steady_state(p::Calibration)::Tuple{Vector,Vector,Vector}` which computes the steady-state of the model computed by hand.__ It returns three vectors, one for the states, one for the controls, one for the shocks. Check that the steady-state satisfies the model equations.


In [17]:
function steady_state(p)

    # ....

    return s, x, e
end

steady_state (generic function with 1 method)

In [18]:
s, x, e = steady_state(p0)

([0.0, 2.920822149964071], [0.29208221499640713], [0.0])

In [ ]:
arbitrage(s,x,s,x,p)

In [ ]:
s - transition(s,x,e,p)

The first order system satisfies:
$$\begin{align}A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0 \\\\ 
s_{t+1} & = & E s_t + F x_t + Q ϵ_t
 \end{align}$$

__Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.__



In [ ]:
struct PerturbedModel
    s::Vector{Float64}
    x::Vector{Float64}
    A::Matrix{Float64}
    B::Matrix{Float64}
    C::Matrix{Float64}
    D::Matrix{Float64}
    E::Matrix{Float64}
    F::Matrix{Float64}
    Q::Matrix{Float64}
end

__Write a function `perturb(s::Vector, x::Vector, p::Calibration)::PerturbedModel`, which returns the first order model, given the steady-state and the calibration. Suggestion: use `ForwardDiff.jl` library.__

In [18]:
using ForwardDiff: jacobian

In [20]:
A = jacobian( u->arbitrage(u, x, s, x, p0), s )
B = jacobian( u->arbitrage(s, u, s, x, p0), x )
C = jacobian( u->arbitrage(s, x, u, x, p0), s )
D = jacobian( u->arbitrage(s, x, s, u, p0), x )
E = jacobian( u->transition(u, x, e, p0), s )
F = jacobian( u->transition(s, u, e, p0), x )
Q = jacobian( u->transition(s, x, u, p0), e )

2×1 Matrix{Float64}:
 0.0
 1.0

In [22]:
function PerturbedModel(p0)
    
    # s, x = steady_state(p0)
    # A = jacobian( u->arbitrage(u, x, s, x, p0), s )
    # B = jacobian( u->arbitrage(s, u, s, x, p0), x )
    # C = jacobian( u->arbitrage(s, x, u, x, p0), s )
    # D = jacobian( u->arbitrage(s, x, s, u, p0), x )
    # E = jacobian( u->transition(u, x, e, p0), s )
    # F = jacobian( u->transition(s, u, e, p0), x )
    # Q = jacobian( u->transition(s, x, u, p0), e )

    return PerturbedModel(

        s,x,A,B,C,D,E,F,Q
    )

end

PerturbedModel

In [25]:
@time m = PerturbedModel(p0)

  0.000067 seconds (50 allocations: 3.234 KiB)


PerturbedModel([0.0, 2.920822149964071], [0.29208221499640713], [5.074626865671642 0.5212190203776081], [-3.679193085018409;;], [-4.938626865671642 -0.5538125831185546], [3.679193085018409;;], [0.9 0.0; 0.0 0.9], [0.0; 1.0;;])

__We look for a linear solution $x_t = X s_t$ . Write the matrix equation which `X` must satisfy. Write a function `residual(X::Array, M::PerturbedModel)` which computes the residual of this equation for a given `X`.__


In [31]:
function residual(X::Matrix{Float64}, M::PerturbedModel)
    Y = X
    A = M.A
    B = M.B
    C = M.C
    D = M.D
    E = M.E
    F = M.F
    R = A + B*X+C*(E+F*X) + D*Y*(E+F*X)
    return R
end

residual (generic function with 1 method)

In [19]:
X0 = [0.1 -0.1; 0.14 0.01]

2×2 Matrix{Float64}:
 0.1   -0.1
 0.14   0.01

In [ ]:
residual(X0, m)

__Write a function `T(X, M::PerturbedModel)`  which implements the time iteration step.__

In [36]:
function T(Y, M::PerturbedModel)

end

T (generic function with 1 method)

__Write function `linear_time_iteration(X_0::Matrix, m::PerturbedModel; maxit=1000, ϵ=1e-8, η=1e-9)::Matrix` which implements the time iteration algorithm. Apply it to `X0 = rand(2,2)` and check that the result satisfies the first order model.__

    

In [20]:
function linear_time_iteration(X_0, m; maxit=1000)
    for k=1:K
        X_1 = T(X_0, m)
    end
end

linear_time_iteration (generic function with 1 method)

__Define two linear operators `L_S(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` and `L_T(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` which implement the derivatives of the simulation and the time-iteration operator respectively.__

In [67]:
L_S(ΔS,X,m) = (m.E + m.F*X)*ΔS

function L_T(ΔY, X,Y, m) 
    - (B+(C+D*Y)*F)\(D*ΔY*(E+F*X))
end


L_T (generic function with 2 methods)

In [79]:
L_T(2*X, X, X,m) - 2*L_T(X, X, X,m)   # check linearity

1×2 Matrix{Float64}:
 0.0  0.0

__Implement a function `spectral_radius(f::Function)::Float64` which implements the power iteration method to compute the biggest eigenvalues of the two previously defined operators. Check that Blanchard-Kahn conditions are met.__

In [73]:
norm(x) = maximum(abs, x)

norm (generic function with 1 method)

In [87]:
function spectral_radius(f, X0; maxit=100)

    ## assumes f is linear

    x0 = rand(size(X0)...)
    u0 = x0 / norm(x0)
    λ = 1.0

    for it=1:maxit
        x1 = f(u0)
        λ = norm(x1)
        println(λ)
        u0 = x1 / λ
    end

    return λ

end

spectral_radius (generic function with 1 method)

In [91]:
spectral_radius(u->L_S(u, X, m), m.s)

1.1610481510375232
1.1086072084230212
1.0745864500379279
1.0507398557507768
1.033104215643956
1.019538168609045
1.0087832500491263
1.0000513811834968
0.9928238752170649
0.9867453900263912
0.9815643198479171
0.9770975144112072
0.9732084799205365
0.9697934113175426
0.9667719589702287
0.964080957685111
0.9616700655203997
0.9594986666204954
0.9575336303511233
0.955747662695045
0.9541180749901054
0.9526258517416164
0.9512549360538716
0.9499916756302723
0.9488243887673399
0.9477430210773227
0.946738871556617
0.9458043721890109
0.9449329092627238
0.9441186774720756
0.9433565599942138
0.9426420293011821
0.9419710646417353
0.9413400830135007
0.9407458811207333
0.9401855863306711
0.9396566150418812
0.9391566371898438
0.9386835458595643
0.9382354311680174
0.9378105577324763
0.9374073451631894
0.9370243511171602
0.9366602565291412
0.9363138527003336
0.9359840299777712
0.9356697678003446
0.9353701259227754
0.9350842366580443
0.9348112980029917
0.9345505675319602
0.9343013569601865
0.934063027292768

0.9290872146546069

In [97]:
# check the result against LinearAlgebra: eigen

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 0.9
 0.9278097265581992
vectors:
2×2 Matrix{Float64}:
  0.0361552  0.0
 -0.999346   1.0

In [88]:
spectral_radius(u->L_T(u, X, X, m), X)

1.22568251832885
1.0875401310195292
1.0188861995073961
0.9778415851174042
0.9505503157337445
0.931099318948722
0.9165402875827734
0.90523851270917
0.8962146223685403
0.8888460984052489
0.8827183116529246
0.8775444277891336
0.8731196938021573
0.8692940369899395
0.8659549536203057
0.8630164447897879
0.8604116448870422
0.858087780635252
0.8560026441039023
0.8541220744990936
0.8524181274329848
0.8508677222373058
0.8494516277622413
0.8481536918004834
0.8469602484889611
0.8458596575078802
0.844841942101573
0.8438985020493184
0.8430218840848323
0.8422055967828365
0.8414439601787578
0.8407319827489524
0.840065260114708
0.8394398911224966
0.8388524079196674
0.8382997173761858
0.8377790517611281
0.8372879270118609
0.8368241072664376
0.8363855745892895
0.8359705030241966
0.8355772362697191
0.8352042684004926
0.834850227160337
0.834513859435622
0.8341940185840038
0.8338896533478273
0.8335997981256987
0.8333235644119937
0.8330601332439176
0.8328087485204202
0.8325687110777595
0.8323393734235806
0.8

0.8275997629892197

In [ ]:
# check the result against LinearAlgebra: eigen (hint: use LinearMaps)

__Write a function `simulate(s0::Vector, X::Matrix, p::Calibration, T::Int64)::Tuple{Matrix, Matrix}` to simulate the model over $T$ periods (by using the formula $\Delta s_t = (E + F X) s_{t-1} + ϵ_t$. Return a matrix for the states (one line per date) and another matrix for the controls. Bonus: add a keyword option to compute variables levels or log-deviations. If possible, return a DataFrame object.__

__Make some nice plots (using *Plots.jl*).__

__Bonus: compute the covariance of the ergodic distribution, using Monte-Carlo simulations. Optimize over
speed and memory consumption. (Check with @time or @benchmark)__